In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import linprog
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
def get_weight(word_list,key_words_list):
    w_8 = [i  for i in word_list if i in key_words_list]
    w_2 = [i  for i in word_list if i not in key_words_list]
        
    if w_8 != [] and w_2 != []:
        net_w = [(0.8/len(w_8)) if i in w_8 else (0.2/len(w_2)) for i in word_list ]
    elif w_8 != []:
        net_w = [(1/len(w_8)) for i in word_list]
    elif w_2 != []:
        net_w = [(1/len(w_2)) for i in word_list]
    else:
        net_w = None
    return net_w

In [3]:
def get_vector(word_list,model):
    doc_vec = []
    for word in word_list:
        doc_vec.append(model[word])
    return doc_vec

In [4]:
def computer_wmd(text_vec,net_vec,text_w,net_w,vec_dim):
    '''
    计算WMD(识别文本-》net)
    '''
    matric_dist = 1 - cosine_similarity(np.array(text_vec),np.array(net_vec))
    c = matric_dist.reshape(-1)
    w = np.array(text_w+net_w)
    
    s1_len = len(text_vec)
    s2_len = len(net_vec)
    
    res = []
    for i in range(s1_len):
        temp = np.zeros((s1_len,s2_len),dtype='float32')
        temp[i:i+1,:] = 1
        res.append(list(temp.reshape(-1)))

    for i in range(s2_len):
        temp = np.zeros((s1_len,s2_len),dtype='float32')
        temp[:,i:i+1] = 1
        res.append(list(temp.reshape(-1))) 
    return linprog(c,A_eq=res,b_eq=w)["fun"]